In [4]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [5]:
from core import get_df
import pandas as pd
import numpy as np

In [6]:
df, failed_runs, not_finished_runs = get_df("01.28", 8)
len(failed_runs)
len(not_finished_runs)
df.shape

2

20

(134, 84)

In [18]:
big_task = df[df["data.nc_first_task"] == 50]
lwf_replay = df[df["data.nc_first_task"] != 50]
lwf_replay = lwf_replay.rename(
    columns={
        # "test_acc": "avg_acc_tag",
        "data.num_tasks": "num_tasks",
        "data.dataset": "dataset",
        "model.network": "network",
        "misc.seed": "seed",
    }
)

In [19]:
all_df = pd.read_csv("all.csv")
df_noreg = all_df[all_df["is_regularized"] == False]

In [20]:
pivot_table = pd.pivot_table(
    big_task[big_task["model.network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["data.dataset", "data.num_tasks"],
    values="avg_acc_tag",
    # aggfunc= lambda x: len(x),
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

data.dataset              cifar100_fixed imagenet_subset_kaggle
data.num_tasks                         6                      6
approach   is_regularized                                      
ewc        False              22.94±0.60             21.03±0.66
           True               28.94±0.59             34.05±0.41
finetuning False              19.54±0.58             18.94±0.31
           True               24.55±0.84             24.78±0.79
lwf        False              42.16±0.80             53.53±0.62
           True               42.67±0.45             54.19±0.25
replay     False              42.69±0.55             47.09±0.89
           True               44.95±0.28             49.58±0.65

In [21]:
pivot_table = pd.pivot_table(
    big_task[big_task["model.network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["data.dataset", "data.num_tasks"],
    values="avg_acc_tag",
    # aggfunc= lambda x: len(x),
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

data.dataset              cifar100_fixed imagenet_subset_kaggle
data.num_tasks                         6                      6
approach   is_regularized                                      
ewc        False              18.09±0.62             16.68±1.22
           True               39.03±0.88             51.32±0.92
finetuning False              19.20±0.97             14.90±1.67
           True               31.09±1.20             38.59±1.20
lwf        False              44.58±0.65                    NaN
           True               44.34±0.56                    NaN
replay     False              36.21±0.74             43.32±0.02
           True               36.62±0.77             44.78±0.79

In [22]:
lwf_replay = pd.concat(
    [df_noreg[df_noreg["approach"].isin(["lwf", "replay"])], lwf_replay], axis=0
)

In [23]:
pivot_table = pd.pivot_table(
    lwf_replay[lwf_replay["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc= lambda x: len(x),
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                 cifar100_fixed                          \
num_tasks                           5           10          20   
approach is_regularized                                          
lwf      False              39.26±0.24  25.45±0.97  16.89±0.73   
         True               37.07±1.74  25.48±0.72  17.12±0.00   
replay   False              40.63±0.89  35.42±2.32  31.60±1.55   
         True               42.13±0.78  37.82±1.12  34.96±0.97   

dataset                 imagenet_subset_kaggle              
num_tasks                                   10          20  
approach is_regularized                                     
lwf      False                      35.47±0.99  19.61±0.38  
         True                       35.66±0.55  19.40±0.34  
replay   False                      38.59±0.72  35.00±1.76  
         True                       39.99±0.34  36.90±0.09

In [24]:
pivot_table = pd.pivot_table(
    lwf_replay[lwf_replay["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc= lambda x: len(x),
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value="",
)
pivot_table

dataset                 cifar100_fixed                          \
num_tasks                           5           10          20   
approach is_regularized                                          
lwf      False              40.70±0.09  29.83±0.21  23.01±0.87   
         True               44.52±0.66  31.38±1.02  21.58±0.38   
replay   False              34.53±0.25  28.59±0.13  25.42±0.25   
         True               34.73±0.85  27.91±0.63  26.06±0.55   

dataset                 imagenet_subset_kaggle              
num_tasks                                   10          20  
approach is_regularized                                     
lwf      False                      43.73±0.46  29.60±0.71  
         True                              NaN         NaN  
replay   False                      34.55±0.86  29.78±0.74  
         True                              NaN         NaN